In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_mwg.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.isocalendar().week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
stock = 'MWG'
df.tail(3)


,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,
MWG,20211208,133.6,135.0,133.0,133.9,688700,2021-12-08,49,2021,2021-49
MWG,20211209,133.9,135.0,133.5,134.0,692300,2021-12-09,49,2021,2021-49
MWG,20211210,133.1,134.9,133.0,134.1,707100,2021-12-10,49,2021,2021-49


In [3]:
df = df.rename(columns={'<CloseFixed>':'<Close>'})
df = df[['<Close>','datetime','<Volume>','Period','Year_no']]

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name=stock))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
MWG,133.9,2021-12-08,688700,2021-49,2021
MWG,134.0,2021-12-09,692300,2021-49,2021
MWG,134.1,2021-12-10,707100,2021-49,2021


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
MWG,9.4948,2014-07-15,13480,2014-29,2014,6.749115
MWG,10.1496,2014-07-16,17290,2014-29,2014,6.896406
MWG,10.8590,2014-07-17,95120,2014-29,2014,6.989438


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    1854.000000
mean        0.169080
std         2.131854
min        -6.999927
25%        -0.846650
50%         0.000000
75%         1.077467
max         6.999945
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    1854.000000
mean       52.084437
std        28.776733
min         9.494800
25%        23.528850
50%        53.747100
75%        67.380500
max       145.000000
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(30).describe()

count    30.000000
mean      0.082709
std       0.112697
min       0.009190
25%       0.021838
50%       0.042505
75%       0.094793
max       0.568890
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean      0.137063
std       0.175415
min       0.013480
25%       0.025895
50%       0.086720
75%       0.124625
max       0.568890
Name: Volumn, dtype: float64

In [16]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
MWG,9.4948,2014-07-15,13480,2014-29,2014,6.749115,0.01348
MWG,10.1496,2014-07-16,17290,2014-29,2014,6.896406,0.01729
MWG,10.8590,2014-07-17,95120,2014-29,2014,6.989438,0.09512
